In [40]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [41]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [42]:
!pip install keras-vit

In [43]:
import torch
import glob
import random

model = torch.hub.load('./yolov5', 'custom', path='/kaggle/input/rsna-breast-cancer-detection-roi-model/rsna-roi-003.pt', source='local')

YOLOv5 🚀 v7.0-365-g12b577c8 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [44]:
def detect_rotation(image):
    """
    Detect the necessary rotation to make the breast appear to be on the left side,
    taking into account images where the breast may be on the bottom or right side.
    """
    # Calculate the sum of pixel intensities along the rows and columns
    row_sum = np.sum(image, axis=1)
    col_sum = np.sum(image, axis=0)

    # Determine if the image is likely in the correct orientation, rotated 90, 180, or 270 degrees
    if np.sum(col_sum[:image.shape[1]//2]) > np.sum(col_sum[image.shape[1]//2:]):
        # More intensity on the left half - likely the breast is already on the left side
        return 0
    elif np.sum(col_sum[image.shape[1]//2:]) > np.sum(col_sum[:image.shape[1]//2]):
        # More intensity on the right half - needs to be rotated by 180 degrees
        return 180
    elif np.sum(row_sum[:image.shape[0]//2]) < np.sum(row_sum[image.shape[0]//2:]):
        # More intensity on the bottom half - likely 270 degrees rotated (i.e., -90 degrees)
        return -90
    elif np.sum(row_sum[:image.shape[0]//2]) > np.sum(row_sum[image.shape[0]//2:]):
        # More intensity on the top half - likely 90 degrees rotated
        return 90
    
    return 0  # Default case (if no clear orientation is detected)

def correct_rotation(image, angle):
    """
    Rotate the image by the specified angle.
    """
    if angle == 0:
        return image  # No rotation needed
    elif angle == 180:
        return cv2.rotate(image, cv2.ROTATE_180)
    elif angle == 90:
        return cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    elif angle == -90 or angle == 270:
        return cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)

In [45]:
# Define the directories and image size
train_directory_no = '/kaggle/input/mammography-private-dataset/0/0'
train_directory_yes = '/kaggle/input/mammography-private-dataset/1/1'
image_size = 320

# MRI Load training dataset
train_data = []
train_labels = []

CLAHE = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(8, 8))

        
for image_name in os.listdir(train_directory_no):
    image_path = os.path.join(train_directory_no, image_name)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    result = model(image)
    result = result.pandas().xyxy[0].to_dict(orient="records")
    if len(result)!=0:
        result = result[0]
        xmin = int(result['xmin'])
        ymin = int(result['ymin'])
        xmax = int(result['xmax'])
        ymax = int(result['ymax'])

        # Crop the image using the bounding box coordinates
        cropped_image = image[ymin:ymax, xmin:xmax]

        detected_angle = detect_rotation(image)
        corrected_image = correct_rotation(cropped_image, detected_angle)

        if corrected_image is not None:
            image_resized = cv2.resize(corrected_image, (image_size, image_size), interpolation=cv2.INTER_AREA)
            image_resized = CLAHE.apply(image_resized)
            image_resized = cv2.equalizeHist(image_resized)
            train_data.append(image_resized)
            train_labels.append(0)
    else:
        detected_angle = detect_rotation(image)
        corrected_image = correct_rotation(image, detected_angle)

        if corrected_image is not None:
            image_resized = cv2.resize(corrected_image, (image_size, image_size), interpolation=cv2.INTER_AREA)
            image_resized = CLAHE.apply(image_resized)
            image_resized = cv2.equalizeHist(image_resized)
            train_data.append(image_resized)
            train_labels.append(0)

for image_name in os.listdir(train_directory_yes):
    image_path = os.path.join(train_directory_yes, image_name)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    result = model(image)
    result = result.pandas().xyxy[0].to_dict(orient="records")
    if len(result)!=0:
        result = result[0]
        xmin = int(result['xmin'])
        ymin = int(result['ymin'])
        xmax = int(result['xmax'])
        ymax = int(result['ymax'])

        # Crop the image using the bounding box coordinates
        cropped_image = image[ymin:ymax, xmin:xmax]

        detected_angle = detect_rotation(image)
        corrected_image = correct_rotation(cropped_image, detected_angle)

        if corrected_image is not None:
            image_resized = cv2.resize(corrected_image, (image_size, image_size), interpolation=cv2.INTER_AREA)
            #image_resized = CLAHE.apply(image_resized)
            image_resized = cv2.equalizeHist(image_resized)
            train_data.append(image_resized)
            train_labels.append(1)
    else:
        detected_angle = detect_rotation(image)
        corrected_image = correct_rotation(image, detected_angle)

        if corrected_image is not None:
            image_resized = cv2.resize(corrected_image, (image_size, image_size), interpolation=cv2.INTER_AREA)
            #image_resized = CLAHE.apply(image_resized)
            image_resized = cv2.equalizeHist(image_resized)
            train_data.append(image_resized)
            train_labels.append(1)

In [46]:
train_data = np.array(train_data) # [0, 255]
train_labels = np.array(train_labels)
train_data = np.repeat(train_data[..., np.newaxis], 3, axis=-1)
print(train_data.shape)
print(train_labels.shape)

(653, 320, 320, 3)
(653,)


In [47]:
from sklearn.utils import shuffle

train_data, train_labels = shuffle(train_data, train_labels, random_state=42)

In [48]:
for i in range(5):
    plt.figure()
    plt.imshow(train_data[i, ..., 0], cmap='gray')
    cv2.imwrite(f"/kaggle/working/{i}.png", train_data[i, ..., 0])
plt.show()

In [49]:
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, CSVLogger

import pickle

from sklearn.metrics import classification_report, confusion_matrix

In [50]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Adding rescaling and validation split

train_gen = datagen.flow(train_data, train_labels, batch_size=8, subset='training')
val_gen = datagen.flow(train_data, train_labels, batch_size=8, subset='validation')

## VGG16

In [51]:
from keras.applications.vgg16 import VGG16

backbone = VGG16(input_shape=(320, 320, 3), include_top=False, weights='imagenet')

In [52]:
backbone.trainable = False

In [53]:
VGG_model = Sequential()
VGG_model.add(backbone)
VGG_model.add(Flatten())
VGG_model.add(Dense(512, activation='relu'))
VGG_model.add(BatchNormalization())
VGG_model.add(Dropout(0.5))
VGG_model.add(Dense(1, activation='sigmoid'))

In [54]:
VGG_model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        metrics=['accuracy']
    )

In [55]:
from keras.callbacks import ModelCheckpoint

# Checkpoint
filepath = "weights.best.keras"  # Change to .keras format
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [56]:
STEP_SIZE_TRAIN = train_gen.n//train_gen.batch_size
STEP_SIZE_VAL = val_gen.n//val_gen.batch_size

In [57]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Fit the model with the callbacks
history = VGG_model.fit(
    train_gen,                  # Your training data generator
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=val_gen,  # Your validation data generator
    validation_steps=STEP_SIZE_VAL,
    epochs=100,
    callbacks=[checkpoint, early_stopping]  # Add the checkpoint and early stopping callbacks
)

VGG_model.save('/kaggle/working/VGG_model.h5')

Epoch 1/100
64/65 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.7004 - loss: 0.6750
Epoch 1: val_accuracy improved from -inf to 0.32812, saving model to weights.best.keras
65/65 ━━━━━━━━━━━━━━━━━━━━ 12s 123ms/step - accuracy: 0.7029 - loss: 0.6704 - val_accuracy: 0.3281 - val_loss: 1.2600
Epoch 2/100
 1/65 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 1.0000 - loss: 0.0935
Epoch 2: val_accuracy improved from 0.32812 to 0.50000, saving model to weights.best.keras
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 1.0000 - loss: 0.0935 - val_accuracy: 0.5000 - val_loss: 0.6263
Epoch 3/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.9053 - loss: 0.2279
Epoch 3: val_accuracy improved from 0.50000 to 0.92969, saving model to weights.best.keras
65/65 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - accuracy: 0.9053 - loss: 0.2280 - val_accuracy: 0.9297 - val_loss: 0.2104
Epoch 4/100
 1/65 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.8750 - loss: 0.2198
Epoch 4: val_accuracy improved from 

In [58]:
from keras.models import load_model
VGG_model = load_model('/kaggle/working/VGG_model.h5')

In [59]:
X_test, y_test = val_gen.x, val_gen.y

In [60]:
score1 = VGG_model.evaluate(X_test, y_test, verbose=0, return_dict=True)
score1

{'accuracy': 0.29230770468711853, 'loss': 32.651336669921875}

In [61]:
y_pred_prob_2 = VGG_model.predict(X_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 196ms/step


In [62]:
threshold = 0.5
y_pred_2 = np.where(y_pred_prob_2 > threshold, 1,0)
y_pred_2.squeeze()

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [63]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred_2)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Normal", "Abnormal"])
disp.plot(cmap=plt.cm.Blues)

plt.title("Confusion Matrix")
plt.show()


In [64]:
cm

array([[ 5, 92],
       [ 0, 33]])

In [65]:
print(classification_report(y_test, y_pred_2))

              precision    recall  f1-score   support

           0       1.00      0.05      0.10        97
           1       0.26      1.00      0.42        33

    accuracy                           0.29       130
   macro avg       0.63      0.53      0.26       130
weighted avg       0.81      0.29      0.18       130



## ViT

In [66]:
!pip install transformers torch torchinfo --quiet


In [69]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("Falah/vit-base-breast-cancer")
model = AutoModelForImageClassification.from_pretrained("Falah/vit-base-breast-cancer")

In [71]:
import torch
model.to(device="cuda")

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [72]:
train_data.shape

(653, 320, 320, 3)

In [88]:
import torch
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification
from tqdm import tqdm

vit_model = model.vit

cls_tokens = []
for i in tqdm(range(len(train_data))):
    
    data_slice_temp = train_data[i]

    #data_slice_temp = torch.tensor(data_slice_temp).to(device="cuda")

    data_slice_temp = Image.fromarray(data_slice_temp.astype(np.uint8))

    inputs = processor(images=data_slice_temp, return_tensors="pt")
    inputs = {key: value.cuda() for key, value in inputs.items()}

    with torch.no_grad():
        outputs = vit_model(**inputs)

    tokens = outputs.last_hidden_state
        
    cls_tokens.append(tokens[:, 0, :].to(device="cpu"))

100%|██████████| 653/653 [00:11<00:00, 59.34it/s]


In [89]:
cls_tokens = np.array(cls_tokens)

In [90]:
cls_tokens.shape

(653, 1, 768)

In [91]:
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, LSTM, Input, GRU, Reshape, Activation, SpatialDropout2D, BatchNormalization, Add, Conv3D, AveragePooling3D, MaxPooling3D
from keras.optimizers import Adam
from sklearn.model_selection import KFold
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
import keras
from keras.layers import TimeDistributed, GlobalMaxPooling3D, GlobalAveragePooling3D, Dense, Dropout, Conv2D, GlobalAveragePooling2D, MaxPooling2D, Flatten, LSTM, Input, GRU, Reshape, Activation, SpatialDropout2D, BatchNormalization, Add

In [92]:
from sklearn.model_selection import train_test_split
data, Xtest, labels, Ytest = train_test_split(cls_tokens, train_labels, test_size=0.2, random_state=42, shuffle=True)
print(Ytest)

[0 1 1 1 0 0 0 1 1 0 0 0 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0]


In [100]:
#%% fit iaaa model
y = labels
#new_data = np.reshape(new_data, (len(new_data), 1, new_data.shape[1], new_data.shape[2]))

accuracies = []
losses = []
precisions = []
recalls = []
aucs = []
mean_of_auc_p_r = []

metrics = [
    'acc',
    keras.metrics.Precision(thresholds=None, top_k=None, class_id=None, name=None, dtype=None),
    keras.metrics.Recall(thresholds=None, top_k=None, class_id=None, name=None, dtype=None),
    keras.metrics.AUC(name='auc')
]

#model = build_tcn(input_shape=(8, new_data.shape[2], new_data.shape[3]))
#model = cnn_bt(input_shape=(new_data.shape[1], new_data.shape[2], new_data.shape[3]))
#model = create_model(new_data.shape[1], new_data.shape[2], new_data.shape[3])
#model = build_3d_cnn(input_shape=(new_data.shape[1], new_data.shape[2], new_data.shape[3], 1)
model2 = Sequential([
    Input(shape=(1, 768)),
    Flatten(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(8, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
    ])
model2.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=metrics)

early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

history = model2.fit(data, labels, epochs=100, batch_size=16, validation_data=(Xtest, Ytest), callbacks=[early_stopping], verbose=1)

scores = model2.evaluate(Xtest, Ytest, verbose=0)
#print(f'Score for fold {fold_no}: {model2.metrics_names[0]} of {scores[0]}; Acc of {scores[1]*100}%; Precision of {scores[2]}; Recall of {scores[3]}; AUC of {scores[4]}\n')
accuracies.append(scores[1])
losses.append(scores[0])
precisions.append(scores[2])
recalls.append(scores[3])
aucs.append(scores[4])
mean_of_auc_p_r.append((scores[2]+scores[3]+scores[4])/3)
print("goal: ", mean_of_auc_p_r[-1])


print(f'Mean accuracy: {np.mean(accuracies)}')
print(f'Mean loss: {np.mean(losses)}')
print(f'Mean of mean of auc, recall and precision: {np.mean(mean_of_auc_p_r)}')

Epoch 1/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 14s 184ms/step - acc: 0.5658 - auc: 0.5997 - loss: 1.0067 - precision_5: 0.3499 - recall_5: 0.5578 - val_acc: 0.7099 - val_auc: 0.6204 - val_loss: 0.6406 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 2/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc: 0.6461 - auc: 0.6915 - loss: 0.7572 - precision_5: 0.4831 - recall_5: 0.6281 - val_acc: 0.7099 - val_auc: 0.6729 - val_loss: 0.6190 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 3/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc: 0.6598 - auc: 0.7359 - loss: 0.7608 - precision_5: 0.4430 - recall_5: 0.7302 - val_acc: 0.7099 - val_auc: 0.6696 - val_loss: 0.6034 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 4/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc: 0.6953 - auc: 0.7605 - loss: 0.6487 - precision_5: 0.4947 - recall_5: 0.7008 - val_acc: 0.7099 - val_auc: 0.7237 - val_loss: 0.5875 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 5/1

In [99]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report

print(Xtest.shape)

predictions = model2.predict(Xtest)
#predictions = model.predict(dtest)

predictions_bool = (predictions > 0.5).astype(int)
print(np.sum(predictions_bool))
from sklearn.metrics import precision_recall_curve, f1_score
print(f1_score(Ytest, predictions_bool))

print(classification_report(Ytest, predictions_bool))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

predictions = predictions.squeeze()

plt.figure(figsize=(12, 8))
x = np.linspace(-10, 10, len(predictions))

idx_sort = predictions.argsort()
predictions = np.sort(predictions)
true_labels = Ytest[idx_sort[::]]


scatter = plt.scatter(range(len(predictions)), predictions, s=2, c=true_labels, cmap='bwr', alpha=0.6)
plt.colorbar(scatter, label='True Labels')
plt.plot(range(len(predictions)), sigmoid(x), label='Sigmoid function')
plt.axhline(y=0.5, color='k', linestyle='--', lw=1, alpha=0.6)

plt.tight_layout()
plt.xlabel('Sample Index')
plt.ylabel('Predicted Probability')
#plt.ylim((0,1))
plt.title('Model Predictions Colored by True Labels')
plt.savefig('/kaggle/working/FIG.png')
plt.show()

(131, 1, 768)
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
44
0.8536585365853658
              precision    recall  f1-score   support

           0       0.97      0.90      0.93        93
           1       0.80      0.92      0.85        38

    accuracy                           0.91       131
   macro avg       0.88      0.91      0.89       131
weighted avg       0.92      0.91      0.91       131

